# **Running the baseline**
<a target="_blank" href="https://colab.research.google.com/github/raphaelrubrice/BornToOverfit/blob/raph/baseline.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/> </a>  

## **Colab setup**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# to avoid having the data on your drive
%cd /content

Mounted at /content/drive
/content


In [ ]:
!git clone https://github.com/raphaelrubrice/BornToOverfit.git
%cd BornToOverfit
!git checkout Camille

Cloning into 'BornToOverfit'...
remote: Enumerating objects: 766, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 766 (delta 56), reused 42 (delta 28), pack-reused 674 (from 1)
Receiving objects: 100% (766/766), 1.08 MiB | 25.72 MiB/s, done.
Resolving deltas: 100% (521/521), done.
/content/BornToOverfit
Branch 'Camille' set up to track remote branch 'Camille' from 'origin'.
Switched to a new branch 'Camille'


Installing requirements

In [ ]:
!pip install -r data_baseline/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 6.2 MB/s eta 0:00:00


Downloading the data

In [ ]:
!cp -r /content/drive/MyDrive/Kaggle_ALTEGRAD/data /content/BornToOverfit/data_baseline/.

cp: cannot open '/content/drive/MyDrive/Kaggle_ALTEGRAD/data/results/baseline.gsheet' for reading: Operation not supported


In [ ]:

import os
import torch
import subprocess
from pathlib import Path

# 1. Configuration des chemins
DATA_DIR = Path("data_baseline/data")

TRAIN_FILE = DATA_DIR / "train_embeddings_RealChemBERT.pt"
VAL_FILE = DATA_DIR / "validation_embeddings_RealChemBERT.pt"


def load_embeddings_pt(filepath):
    print(f"📥 Chargement de {filepath}...")

    data = torch.load(filepath, map_location='cpu')

    # On récupère les clés qu'on a définies lors de la génération
    ids = data['ids']
    embeddings_tensor = data['embeddings']

    print(f"✅ Chargé ! Shape: {embeddings_tensor.shape}")
    return ids, embeddings_tensor

# 2. Logique Principale
if TRAIN_FILE.exists() and VAL_FILE.exists():
    print("🚀 Fichiers .pt trouvés ! Chargement direct...")

    train_ids, train_embeddings = load_embeddings_pt(TRAIN_FILE)
    val_ids, val_embeddings = load_embeddings_pt(VAL_FILE)

    print(f"Premier ID train : {train_ids[0]}")
    print(f"Norme du premier vecteur : {train_embeddings[0].norm().item():.4f}")

else:
    print(f" Fichiers non trouvés : {TRAIN_FILE}")
    print("⚙️ Lancement du script de génération...")

    cmd = [
        "python", "data_baseline/generate_description_embeddings_automatic.py",
        "--data_dir", "data_baseline/data",
        "--model_name", "recobo/chemical-bert-uncased",
        "--pooling", "cls",
        "--splits", "train", "validation"
    ]

    process = subprocess.run(cmd, capture_output=False)

    if process.returncode == 0:
        print("\n Génération terminée avec succès !")
    else:
        print("\n Erreur lors de la génération.")

🚀 Fichiers .pt trouvés ! Chargement direct...
📥 Chargement de data_baseline/data/train_embeddings_RealChemBERT.pt...
✅ Chargé ! Shape: torch.Size([31008, 768])
📥 Chargement de data_baseline/data/validation_embeddings_RealChemBERT.pt...
✅ Chargé ! Shape: torch.Size([1000, 768])
Premier ID train : 0
Norme du premier vecteur : 24.6847


## **Dual Encoder**

In [ ]:
# Avec backup automatique sur Drive (recommandé)
!python data_baseline/train_dual_encoder_optimized.py \
    --data_dir data_baseline/data \
    --drive_backup_dir /content/drive/MyDrive/Altegrad_Results/dual_encoder_checkpoints \
    --lr_gnn 8e-4 \
    --lr_bert 3e-5 \
    --weight_decay 1e-4 \
    --freeze_layers 0 \
    --margin 0.2 \
    --batch_size 16 \
    --grad_accum 8 \
    --epochs 150 \
    --final_training

🚀 DUAL ENCODER TRAINING - FINAL (train+val)
Device      : cuda
☁️ Google Drive: MONTÉ (/content/drive/MyDrive/Altegrad_Results/dual_encoder_checkpoints)
LR GNN      : 0.0008
LR BERT     : 3e-05
Weight Decay: 0.0001
Freeze Layers: 0
Margin      : 0.2
Batch Size  : 16 × 8 = 128 (effective)
Epochs      : 150
tokenizer_config.json: 100% 339/339 [00:00<00:00, 2.72MB/s]
config.json: 100% 612/612 [00:00<00:00, 4.96MB/s]
vocab.txt: 228kB [00:00, 40.8MB/s]
special_tokens_map.json: 100% 112/112 [00:00<00:00, 1.25MB/s]
📊 Training sur 32008 exemples (train: 31008, val: 1000)
2026-01-12 11:34:24.736229: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-12 11:34:24.758838: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting 

___

___

In [ ]:
!python -m pip -q install -U evaluate bert-score sacrebleu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 12.8 MB/s eta 0:00:00


# Code pour avoir les résultats et convertir au format csv pour kaggle

In [ ]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch_geometric.data import Batch
from torch_geometric.nn import GPSConv, GINEConv, global_add_pool
from transformers import AutoModel, AutoTokenizer
import evaluate
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# ==============================================================================
# 0. CONFIGURATION
# ==============================================================================
#
#MODEL_PATH = "data_baseline/data/dual_FINAL_RESUMED_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128_ep250.pt"
MODEL_PATH = "/content/drive/MyDrive/Altegrad_Results/dual_encoder_checkpoints/dual_FINAL_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128_ep150.pt"
DATA_DIR = Path("data_baseline/data")
MODEL_NAME = "recobo/chemical-bert-uncased"
BATCH_SIZE = 32
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Output
SUBMISSION_PATH = "submission_dual_encoder_final.csv"

print(f"⚙️ Configuration : {DEVICE}")

# ==============================================================================
# 1. DÉFINITION DE L'ARCHITECTURE
# ==============================================================================
ATOM_DIMS = [119, 4, 11, 12, 9, 5, 8, 2, 2]
BOND_DIMS = [22, 6, 2]

class AtomEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in ATOM_DIMS])
    def forward(self, x):
        return sum(emb(x[:, i]) for i, emb in enumerate(self.embeddings))

class BondEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in BOND_DIMS])
    def forward(self, edge_attr):
        return sum(emb(edge_attr[:, i]) for i, emb in enumerate(self.embeddings))

class MolGNN(nn.Module):
    def __init__(self, hidden_dim=256, out_dim=768, num_layers=4, num_heads=8, dropout=0.1):
        super().__init__()
        self.atom_encoder = AtomEncoder(hidden_dim)
        self.bond_encoder = BondEncoder(hidden_dim)
        self.convs = nn.ModuleList()
        for _ in range(num_layers):
            local_nn = nn.Sequential(
                nn.Linear(hidden_dim, 2 * hidden_dim),
                nn.BatchNorm1d(2 * hidden_dim),
                nn.ReLU(),
                nn.Linear(2 * hidden_dim, hidden_dim),
            )
            self.convs.append(GPSConv(
                hidden_dim,
                GINEConv(local_nn, train_eps=True, edge_dim=hidden_dim),
                heads=num_heads,
                dropout=dropout,
                attn_type='multihead'
            ))
        self.pool = global_add_pool
        self.proj = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, batch):
        h = self.atom_encoder(batch.x)
        edge_attr = self.bond_encoder(batch.edge_attr)
        for conv in self.convs:
            h = conv(h, batch.edge_index, batch.batch, edge_attr=edge_attr)
        return self.proj(self.pool(h, batch.batch))

class DualEncoder(nn.Module):
    def __init__(self, model_name, gnn_args, freeze_layers=0):
        super().__init__()
        self.graph_encoder = MolGNN(**gnn_args)
        self.text_encoder = AutoModel.from_pretrained(model_name)

        if freeze_layers > 0:
            print(f"❄️ Gel des {freeze_layers} premières couches de BERT")
            for param in self.text_encoder.embeddings.parameters():
                param.requires_grad = False
            for i in range(freeze_layers):
                if i < len(self.text_encoder.encoder.layer):
                    for param in self.text_encoder.encoder.layer[i].parameters():
                        param.requires_grad = False

        bert_dim = self.text_encoder.config.hidden_size
        out_dim = gnn_args['out_dim']
        self.text_proj = nn.Linear(bert_dim, out_dim) if bert_dim != out_dim else nn.Identity()

    def forward_text(self, input_ids, attention_mask):
        t_out = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        return F.normalize(self.text_proj(t_out.last_hidden_state[:, 0, :]), dim=-1)

    def forward_graph(self, batch):
        return F.normalize(self.graph_encoder(batch), dim=-1)

# ==============================================================================
# 2. FONCTIONS DE CHARGEMENT
# ==============================================================================
GNN_CONF_FORCE = {
    'hidden_dim': 256,
    'out_dim': 768,
    'num_layers': 4,
    'num_heads': 8,
    'dropout': 0.1
}

# 2. MODIFICATION DE LA FONCTION DE CHARGEMENT
def load_dual_encoder(checkpoint_path, device='cuda'):
    """Charge le DualEncoder avec la config forcée de ton entraînement."""

    print(f"📂 Lecture du checkpoint : {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)

    # Récupération du state_dict (poids)
    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        state_dict = checkpoint['model_state_dict']
        # On ignore les args sauvegardés pour utiliser ceux qu'on est sûrs d'avoir utilisés
        print(f"   ℹ️ Info Checkpoint - Epoch: {checkpoint.get('epoch', '?')}, Loss: {checkpoint.get('train_loss', '?')}")
    else:
        state_dict = checkpoint
        print(f"   ⚠️ Format ancien ou direct state_dict détecté")

    # Initialisation du modèle avec LA BONNE CONFIGURATION
    # On utilise GNN_CONF_FORCE défini plus haut
    model = DualEncoder(MODEL_NAME, GNN_CONF_FORCE, freeze_layers=0)

    # Chargement des poids
    # strict=False peut être utile si tu as des soucis de préfixes "module.",

    try:
        model.load_state_dict(state_dict)
        print("✅ Poids chargés avec succès (strict=True)")
    except RuntimeError as e:
        print(f"⚠️ Erreur de chargement stricte, tentative avec strict=False... ({e})")
        model.load_state_dict(state_dict, strict=False)
        print("✅ Poids chargés (strict=False)")

    model.to(device)
    model.eval()

    return model, GNN_CONF_FORCE


def load_texts_and_graphs(pkl_path, has_description=True):
    """
    Charge les graphes et extrait les descriptions (si disponibles).

    Args:
        pkl_path: Chemin vers le fichier pickle
        has_description: Si False, retourne None pour les textes
    """
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)

    if has_description:
        texts = [d.description for d in data]
    else:
        texts = None  # Pas de description dans le test set

    return data, texts


class GraphDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs
    def __len__(self):
        return len(self.graphs)
    def __getitem__(self, idx):
        return self.graphs[idx]


def collate_text(batch):
    return batch


def collate_graph(batch):
    return Batch.from_data_list(batch)


# ==============================================================================
# 3. MOTEUR D'INFERENCE
# ==============================================================================
@torch.no_grad()
def generate_reference_embeddings(model, tokenizer, texts, device, batch_size=32):
    """Recalcule les embeddings avec le modèle fine-tuné."""
    print(f"🔄 Recalcul des embeddings de référence ({len(texts)} textes)...")
    model.eval()
    embeddings = []

    loader = DataLoader(texts, batch_size=batch_size, collate_fn=collate_text)

    for batch_texts in tqdm(loader, desc="Encoding Reference Texts"):
        inputs = tokenizer(
            batch_texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=128
        ).to(device)

        emb = model.forward_text(inputs['input_ids'], inputs['attention_mask'])
        embeddings.append(emb.cpu())

    return torch.cat(embeddings, dim=0)


@torch.no_grad()
def encode_graphs(model, graphs, device, batch_size=32):
    """Encode les graphes en embeddings."""
    print(f"🧪 Encodage de {len(graphs)} graphes...")
    model.eval()
    embeddings = []

    loader = DataLoader(
        GraphDataset(graphs),
        batch_size=batch_size,
        collate_fn=collate_graph
    )

    for batch in tqdm(loader, desc="Encoding Graphs"):
        batch = batch.to(device)
        emb = model.forward_graph(batch)
        embeddings.append(emb.cpu())

    return torch.cat(embeddings, dim=0)


def generate_submission(predictions, output_path):
    """
    Génère le fichier de submission Kaggle.

    Args:
        predictions: Liste de textes prédits (dans l'ordre du test set)
        output_path: Chemin du fichier CSV à créer
    """
    submission_df = pd.DataFrame({
        'ID': range(len(predictions)),
        'description': predictions
    })

    submission_df.to_csv(output_path, index=False)
    print(f"\n💾 Submission sauvegardée : {output_path}")
    print(f"   Nombre de prédictions : {len(predictions)}")
    print(f"   Preview :")
    print(submission_df.head(10).to_string(index=False))


@torch.no_grad()
def run_evaluation_and_submission():
    """Évaluation + Génération de submission Kaggle."""

    # === 1. CHARGEMENT MODÈLE ===
    print(f"📥 Chargement du Dual Encoder depuis {MODEL_PATH}")
    model, config = load_dual_encoder(MODEL_PATH, DEVICE)

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # === 2. CHARGEMENT DONNÉES ===
    print("📂 Chargement des datasets...")

    # 🆕 CHANGEMENT MAJEUR : Combiner train + val pour la référence
    train_graphs, train_texts = load_texts_and_graphs(DATA_DIR / "train_graphs.pkl")
    val_graphs, val_texts = load_texts_and_graphs(DATA_DIR / "validation_graphs.pkl")

    # Combiner train + val
    reference_graphs = train_graphs + val_graphs
    reference_texts = train_texts + val_texts

    print(f"   Train: {len(train_graphs)} exemples")
    print(f"   Val:   {len(val_graphs)} exemples")
    print(f"   📚 RÉFÉRENCE (train+val): {len(reference_texts)} exemples")

    # Test set pour Kaggle
    test_graphs, _ = load_texts_and_graphs(DATA_DIR / "test_graphs.pkl", has_description=False)
    print(f"   Test:  {len(test_graphs)} exemples (pour Kaggle)")

    # === 3. CRÉATION BANQUE DE RÉFÉRENCE (TRAIN+VAL) ===
    print("\n" + "=" * 70)
    print("🔄 ENCODAGE DE LA BANQUE DE RÉFÉRENCE (TRAIN+VAL)")
    print("=" * 70)

    reference_embs = generate_reference_embeddings(
        model, tokenizer, reference_texts, DEVICE, batch_size=BATCH_SIZE
    )
    reference_embs = reference_embs.to(DEVICE)

    print(f"   ✅ Embeddings référence: {reference_embs.shape}")

    # === 4. GÉNÉRATION PRÉDICTIONS TEST (pour Kaggle) ===
    print("\n" + "=" * 70)
    print("🎯 GÉNÉRATION SUBMISSION KAGGLE (TEST SET)")
    print("=" * 70)

    test_mol_embs = encode_graphs(model, test_graphs, DEVICE, batch_size=BATCH_SIZE)
    test_mol_embs = test_mol_embs.to(DEVICE)

    print("🔍 Recherche des descriptions les plus proches (Test)...")
    sims_test = test_mol_embs @ reference_embs.t()
    best_indices_test = sims_test.argmax(dim=-1).cpu().numpy()

    preds_test = [reference_texts[idx] for idx in best_indices_test]

    # Génération du fichier submission
    generate_submission(preds_test, SUBMISSION_PATH)

    # Exemples de prédictions
    print("\n📝 EXEMPLES DE PRÉDICTIONS (TEST SET) :")
    print("-" * 70)
    for i in range(min(5, len(preds_test))):
        print(f"\nId {i}:")
        print(f"  {preds_test[i][:120]}...")
    print("-" * 70)

    return {
        'submission_file': SUBMISSION_PATH,
        'num_references': len(reference_texts),
        'num_predictions': len(preds_test)
    }


if __name__ == "__main__":
    results = run_evaluation_and_submission()

    print("\n" + "=" * 70)
    print("✅ TERMINÉ !")
    print("=" * 70)
    print(f"📄 Fichier de submission : {results['submission_file']}")
    print(f"📚 Banque de référence   : {results['num_references']} descriptions")
    print(f"🎯 Prédictions générées  : {results['num_predictions']}")
    print("=" * 70)
    print("\n💡 go Upload le fichier sur Kaggle !")

⚙️ Configuration : cuda
📥 Chargement du Dual Encoder depuis /content/drive/MyDrive/Altegrad_Results/dual_encoder_checkpoints/dual_FINAL_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128_ep150.pt
📂 Lecture du checkpoint : /content/drive/MyDrive/Altegrad_Results/dual_encoder_checkpoints/dual_FINAL_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128_ep150.pt
   ℹ️ Info Checkpoint - Epoch: 150, Loss: 0.0013596335987279798


Some weights of BertModel were not initialized from the model checkpoint at recobo/chemical-bert-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Poids chargés avec succès (strict=True)
📂 Chargement des datasets...
   Train: 31008 exemples
   Val:   1000 exemples
   📚 RÉFÉRENCE (train+val): 32008 exemples
   Test:  1000 exemples (pour Kaggle)

🔄 ENCODAGE DE LA BANQUE DE RÉFÉRENCE (TRAIN+VAL)
🔄 Recalcul des embeddings de référence (32008 textes)...


Encoding Reference Texts: 100%|██████████| 1001/1001 [01:46<00:00,  9.38it/s]


   ✅ Embeddings référence: torch.Size([32008, 768])

🎯 GÉNÉRATION SUBMISSION KAGGLE (TEST SET)
🧪 Encodage de 1000 graphes...


Encoding Graphs: 100%|██████████| 32/32 [00:00<00:00, 70.68it/s]


🔍 Recherche des descriptions les plus proches (Test)...

💾 Submission sauvegardée : submission_dual_encoder_final.csv
   Nombre de prédictions : 1000
   Preview :
 ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         description
  0                                                                                                                                                                                                                                                                                                  The molecule is a glycodihydroceramide that is a sphinga

In [ ]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch_geometric.data import Batch
from torch_geometric.nn import GPSConv, GINEConv, global_add_pool
from transformers import AutoModel, AutoTokenizer
import evaluate
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# ==============================================================================
# 0. CONFIGURATION
# ==============================================================================
#MODEL_PATH = "data_baseline/data/dual_FINAL_RESUMED_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128_ep250.pt"
MODEL_PATH = "/content/drive/MyDrive/Altegrad_Results/dual_encoder_checkpoints_head4/dual_FINAL_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128_ep150.pt"
DATA_DIR = Path("data_baseline/data")
MODEL_NAME = "recobo/chemical-bert-uncased"
BATCH_SIZE = 32
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Output
SUBMISSION_PATH = "submission_dual_encoder_final.csv"

print(f"⚙️ Configuration : {DEVICE}")

# ==============================================================================
# 1. DÉFINITION DE L'ARCHITECTURE
# ==============================================================================
ATOM_DIMS = [119, 4, 11, 12, 9, 5, 8, 2, 2]
BOND_DIMS = [22, 6, 2]

class AtomEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in ATOM_DIMS])
    def forward(self, x):
        return sum(emb(x[:, i]) for i, emb in enumerate(self.embeddings))

class BondEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in BOND_DIMS])
    def forward(self, edge_attr):
        return sum(emb(edge_attr[:, i]) for i, emb in enumerate(self.embeddings))

class MolGNN(nn.Module):
    def __init__(self, hidden_dim=256, out_dim=768, num_layers=4, num_heads=4, dropout=0.1):
        super().__init__()
        self.atom_encoder = AtomEncoder(hidden_dim)
        self.bond_encoder = BondEncoder(hidden_dim)
        self.convs = nn.ModuleList()
        for _ in range(num_layers):
            local_nn = nn.Sequential(
                nn.Linear(hidden_dim, 2 * hidden_dim),
                nn.BatchNorm1d(2 * hidden_dim),
                nn.ReLU(),
                nn.Linear(2 * hidden_dim, hidden_dim),
            )
            self.convs.append(GPSConv(
                hidden_dim,
                GINEConv(local_nn, train_eps=True, edge_dim=hidden_dim),
                heads=num_heads,
                dropout=dropout,
                attn_type='multihead'
            ))
        self.pool = global_add_pool
        self.proj = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, batch):
        h = self.atom_encoder(batch.x)
        edge_attr = self.bond_encoder(batch.edge_attr)
        for conv in self.convs:
            h = conv(h, batch.edge_index, batch.batch, edge_attr=edge_attr)
        return self.proj(self.pool(h, batch.batch))

class DualEncoder(nn.Module):
    def __init__(self, model_name, gnn_args, freeze_layers=0):
        super().__init__()
        self.graph_encoder = MolGNN(**gnn_args)
        self.text_encoder = AutoModel.from_pretrained(model_name)

        if freeze_layers > 0:
            print(f"❄️ Gel des {freeze_layers} premières couches de BERT")
            for param in self.text_encoder.embeddings.parameters():
                param.requires_grad = False
            for i in range(freeze_layers):
                if i < len(self.text_encoder.encoder.layer):
                    for param in self.text_encoder.encoder.layer[i].parameters():
                        param.requires_grad = False

        bert_dim = self.text_encoder.config.hidden_size
        out_dim = gnn_args['out_dim']
        self.text_proj = nn.Linear(bert_dim, out_dim) if bert_dim != out_dim else nn.Identity()

    def forward_text(self, input_ids, attention_mask):
        t_out = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        return F.normalize(self.text_proj(t_out.last_hidden_state[:, 0, :]), dim=-1)

    def forward_graph(self, batch):
        return F.normalize(self.graph_encoder(batch), dim=-1)

# ==============================================================================
# 2. FONCTIONS DE CHARGEMENT
# ==============================================================================
GNN_CONF_FORCE = {
    'hidden_dim': 256,
    'out_dim': 768,
    'num_layers': 4,
    'num_heads': 4,
    'dropout': 0.1
}

# 2.
def load_dual_encoder(checkpoint_path, device='cuda'):
    """Charge le DualEncoder avec la config forcée de ton entraînement."""

    print(f"📂 Lecture du checkpoint : {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)

    # Récupération du state_dict (poids)
    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        state_dict = checkpoint['model_state_dict']
        print(f"   ℹ️ Info Checkpoint - Epoch: {checkpoint.get('epoch', '?')}, Loss: {checkpoint.get('train_loss', '?')}")
    else:
        state_dict = checkpoint
        print(f"   ⚠️ Format ancien ou direct state_dict détecté")

    # Initialisation du modèle avec LA BONNE CONFIGURATION
    # On utilise GNN_CONF_FORCE défini plus haut
    model = DualEncoder(MODEL_NAME, GNN_CONF_FORCE, freeze_layers=0)

    # Chargement des poids
    try:
        model.load_state_dict(state_dict)
        print("✅ Poids chargés avec succès (strict=True)")
    except RuntimeError as e:
        print(f"⚠️ Erreur de chargement stricte, tentative avec strict=False... ({e})")
        model.load_state_dict(state_dict, strict=False)
        print("✅ Poids chargés (strict=False)")

    model.to(device)
    model.eval()

    return model, GNN_CONF_FORCE


def load_texts_and_graphs(pkl_path, has_description=True):
    """
    Charge les graphes et extrait les descriptions (si disponibles).

    Args:
        pkl_path: Chemin vers le fichier pickle
        has_description: Si False, retourne None pour les textes
    """
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)

    if has_description:
        texts = [d.description for d in data]
    else:
        texts = None  # Pas de description dans le test set

    return data, texts


class GraphDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs
    def __len__(self):
        return len(self.graphs)
    def __getitem__(self, idx):
        return self.graphs[idx]


def collate_text(batch):
    return batch


def collate_graph(batch):
    return Batch.from_data_list(batch)


# ==============================================================================
# 3. MOTEUR D'INFERENCE
# ==============================================================================
@torch.no_grad()
def generate_reference_embeddings(model, tokenizer, texts, device, batch_size=32):
    """Recalcule les embeddings avec le modèle fine-tuné."""
    print(f"🔄 Recalcul des embeddings de référence ({len(texts)} textes)...")
    model.eval()
    embeddings = []

    loader = DataLoader(texts, batch_size=batch_size, collate_fn=collate_text)

    for batch_texts in tqdm(loader, desc="Encoding Reference Texts"):
        inputs = tokenizer(
            batch_texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=128
        ).to(device)

        emb = model.forward_text(inputs['input_ids'], inputs['attention_mask'])
        embeddings.append(emb.cpu())

    return torch.cat(embeddings, dim=0)


@torch.no_grad()
def encode_graphs(model, graphs, device, batch_size=32):
    """Encode les graphes en embeddings."""
    print(f"🧪 Encodage de {len(graphs)} graphes...")
    model.eval()
    embeddings = []

    loader = DataLoader(
        GraphDataset(graphs),
        batch_size=batch_size,
        collate_fn=collate_graph
    )

    for batch in tqdm(loader, desc="Encoding Graphs"):
        batch = batch.to(device)
        emb = model.forward_graph(batch)
        embeddings.append(emb.cpu())

    return torch.cat(embeddings, dim=0)


def generate_submission(predictions, output_path):
    """
    Génère le fichier de submission Kaggle.

    Args:
        predictions: Liste de textes prédits (dans l'ordre du test set)
        output_path: Chemin du fichier CSV à créer
    """
    submission_df = pd.DataFrame({
        'ID': range(len(predictions)),
        'description': predictions
    })

    submission_df.to_csv(output_path, index=False)
    print(f"\n💾 Submission sauvegardée : {output_path}")
    print(f"   Nombre de prédictions : {len(predictions)}")
    print(f"   Preview :")
    print(submission_df.head(10).to_string(index=False))


@torch.no_grad()
def run_evaluation_and_submission():
    """Évaluation + Génération de submission Kaggle."""

    # === 1. CHARGEMENT MODÈLE ===
    print(f"📥 Chargement du Dual Encoder depuis {MODEL_PATH}")
    model, config = load_dual_encoder(MODEL_PATH, DEVICE)

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # === 2. CHARGEMENT DONNÉES ===
    print("📂 Chargement des datasets...")

    # 🆕 CHANGEMENT MAJEUR : Combiner train + val pour la référence
    train_graphs, train_texts = load_texts_and_graphs(DATA_DIR / "train_graphs.pkl")
    val_graphs, val_texts = load_texts_and_graphs(DATA_DIR / "validation_graphs.pkl")

    # Combiner train + val
    reference_graphs = train_graphs + val_graphs
    reference_texts = train_texts + val_texts

    print(f"   Train: {len(train_graphs)} exemples")
    print(f"   Val:   {len(val_graphs)} exemples")
    print(f"   📚 RÉFÉRENCE (train+val): {len(reference_texts)} exemples")

    # Test set pour Kaggle
    test_graphs, _ = load_texts_and_graphs(DATA_DIR / "test_graphs.pkl", has_description=False)
    print(f"   Test:  {len(test_graphs)} exemples (pour Kaggle)")

    # === 3. CRÉATION BANQUE DE RÉFÉRENCE (TRAIN+VAL) ===
    print("\n" + "=" * 70)
    print("🔄 ENCODAGE DE LA BANQUE DE RÉFÉRENCE (TRAIN+VAL)")
    print("=" * 70)

    reference_embs = generate_reference_embeddings(
        model, tokenizer, reference_texts, DEVICE, batch_size=BATCH_SIZE
    )
    reference_embs = reference_embs.to(DEVICE)

    print(f"   ✅ Embeddings référence: {reference_embs.shape}")

    # === 4. GÉNÉRATION PRÉDICTIONS TEST (pour Kaggle) ===
    print("\n" + "=" * 70)
    print("🎯 GÉNÉRATION SUBMISSION KAGGLE (TEST SET)")
    print("=" * 70)

    test_mol_embs = encode_graphs(model, test_graphs, DEVICE, batch_size=BATCH_SIZE)
    test_mol_embs = test_mol_embs.to(DEVICE)

    print("🔍 Recherche des descriptions les plus proches (Test)...")
    sims_test = test_mol_embs @ reference_embs.t()
    best_indices_test = sims_test.argmax(dim=-1).cpu().numpy()

    preds_test = [reference_texts[idx] for idx in best_indices_test]

    # Génération du fichier submission
    generate_submission(preds_test, SUBMISSION_PATH)

    # Exemples de prédictions
    print("\n📝 EXEMPLES DE PRÉDICTIONS (TEST SET) :")
    print("-" * 70)
    for i in range(min(5, len(preds_test))):
        print(f"\nId {i}:")
        print(f"  {preds_test[i][:120]}...")
    print("-" * 70)

    return {
        'submission_file': SUBMISSION_PATH,
        'num_references': len(reference_texts),
        'num_predictions': len(preds_test)
    }


if __name__ == "__main__":
    results = run_evaluation_and_submission()

    print("\n" + "=" * 70)
    print("✅ TERMINÉ !")
    print("=" * 70)
    print(f"📄 Fichier de submission : {results['submission_file']}")
    print(f"📚 Banque de référence   : {results['num_references']} descriptions")
    print(f"🎯 Prédictions générées  : {results['num_predictions']}")
    print("=" * 70)
    print("\n💡 Prochaine étape : Upload le fichier sur Kaggle !")

⚙️ Configuration : cuda
📥 Chargement du Dual Encoder depuis /content/drive/MyDrive/Altegrad_Results/dual_encoder_checkpoints_head4/dual_FINAL_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128_ep150.pt
📂 Lecture du checkpoint : /content/drive/MyDrive/Altegrad_Results/dual_encoder_checkpoints_head4/dual_FINAL_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128_ep150.pt
   ℹ️ Info Checkpoint - Epoch: 150, Loss: 0.0015605676775869877


Some weights of BertModel were not initialized from the model checkpoint at recobo/chemical-bert-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Poids chargés avec succès (strict=True)
📂 Chargement des datasets...
   Train: 31008 exemples
   Val:   1000 exemples
   📚 RÉFÉRENCE (train+val): 32008 exemples
   Test:  1000 exemples (pour Kaggle)

🔄 ENCODAGE DE LA BANQUE DE RÉFÉRENCE (TRAIN+VAL)
🔄 Recalcul des embeddings de référence (32008 textes)...


Encoding Reference Texts: 100%|██████████| 1001/1001 [01:45<00:00,  9.47it/s]


   ✅ Embeddings référence: torch.Size([32008, 768])

🎯 GÉNÉRATION SUBMISSION KAGGLE (TEST SET)
🧪 Encodage de 1000 graphes...


Encoding Graphs: 100%|██████████| 32/32 [00:00<00:00, 75.26it/s]


🔍 Recherche des descriptions les plus proches (Test)...

💾 Submission sauvegardée : submission_dual_encoder_final.csv
   Nombre de prédictions : 1000
   Preview :
 ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         description
  0                                                                                                                                                                                                                                                                                                                         The molecule is a glycophytoceram

In [ ]:
%%writefile data_baseline/resume_training_dual_encoder.py
import os
import argparse
import pickle
import json
import shutil
from pathlib import Path
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR

from torch_geometric.data import Batch
from torch_geometric.nn import GPSConv, GINEConv, global_add_pool
from transformers import AutoModel, AutoTokenizer

# --- DATASET ---
class RawTextGraphDataset(Dataset):
    def __init__(self, pkl_path):
        self.pkl_path = Path(pkl_path)
        with open(self.pkl_path, 'rb') as f:
            self.data_list = pickle.load(f)
    def __len__(self): return len(self.data_list)
    def __getitem__(self, idx):
        data = self.data_list[idx]
        text = data.description if hasattr(data, 'description') else ""
        return data, text

class DualCollate:
    def __init__(self, tokenizer, max_len=128):
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __call__(self, batch):
        graphs, texts = zip(*batch)
        batched_graphs = Batch.from_data_list(graphs)
        text_inputs = self.tokenizer(list(texts), padding=True, truncation=True,
                                     max_length=self.max_len, return_tensors="pt")
        return batched_graphs, text_inputs

# --- MODELS ---
ATOM_DIMS = [119, 4, 11, 12, 9, 5, 8, 2, 2]
BOND_DIMS = [22, 6, 2]

class AtomEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in ATOM_DIMS])
    def forward(self, x):
        return sum(emb(x[:, i]) for i, emb in enumerate(self.embeddings))

class BondEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in BOND_DIMS])
    def forward(self, edge_attr):
        return sum(emb(edge_attr[:, i]) for i, emb in enumerate(self.embeddings))

class MolGNN(nn.Module):
    def __init__(self, hidden_dim=256, out_dim=768, num_layers=4, num_heads=8, dropout=0.1):
        super().__init__()
        self.atom_encoder = AtomEncoder(hidden_dim)
        self.bond_encoder = BondEncoder(hidden_dim)
        self.convs = nn.ModuleList()
        for _ in range(num_layers):
            local_nn = nn.Sequential(
                nn.Linear(hidden_dim, 2 * hidden_dim),
                nn.BatchNorm1d(2 * hidden_dim),
                nn.ReLU(),
                nn.Linear(2 * hidden_dim, hidden_dim),
            )
            self.convs.append(GPSConv(
                hidden_dim,
                GINEConv(local_nn, train_eps=True, edge_dim=hidden_dim),
                heads=num_heads,
                dropout=dropout,
                attn_type='multihead'
            ))
        self.pool = global_add_pool
        self.proj = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, batch):
        h = self.atom_encoder(batch.x)
        edge_attr = self.bond_encoder(batch.edge_attr)
        for conv in self.convs:
            h = conv(h, batch.edge_index, batch.batch, edge_attr=edge_attr)
        return self.proj(self.pool(h, batch.batch))

class DualEncoder(nn.Module):
    def __init__(self, model_name, gnn_args, freeze_layers=0):
        super().__init__()
        self.graph_encoder = MolGNN(**gnn_args)
        self.text_encoder = AutoModel.from_pretrained(model_name)

        if freeze_layers > 0:
            print(f"❄️ Gel des {freeze_layers} premières couches de BERT")
            for param in self.text_encoder.embeddings.parameters():
                param.requires_grad = False
            for i in range(freeze_layers):
                if i < len(self.text_encoder.encoder.layer):
                    for param in self.text_encoder.encoder.layer[i].parameters():
                        param.requires_grad = False

        bert_dim = self.text_encoder.config.hidden_size
        out_dim = gnn_args['out_dim']
        self.text_proj = nn.Linear(bert_dim, out_dim) if bert_dim != out_dim else nn.Identity()

    def forward(self, batch_graphs, text_inputs):
        g_emb = self.graph_encoder(batch_graphs)
        t_out = self.text_encoder(**text_inputs)
        t_emb = self.text_proj(t_out.last_hidden_state[:, 0, :])
        return F.normalize(g_emb, dim=-1), F.normalize(t_emb, dim=-1)

# --- LOSS ---
def triplet_loss(mol_vec, txt_vec, margin=0.2):
    sims = mol_vec @ txt_vec.t()
    mask = torch.eye(sims.size(0), device=sims.device).bool()
    min_value = torch.finfo(sims.dtype).min
    neg_sims = sims.masked_fill(mask, min_value)

    hard_neg_m2t = neg_sims.max(dim=1, keepdim=True)[0]
    hard_neg_t2m = neg_sims.max(dim=0, keepdim=True)[0].t()
    pos_sims = sims.diag().unsqueeze(1)

    loss = (F.relu(margin + hard_neg_m2t - pos_sims).mean() +
            F.relu(margin + hard_neg_t2m - pos_sims).mean())
    return loss / 2

# --- BACKUP FONCTION ---
def backup_to_drive(file_path, drive_backup_dir):
    """Copie automatique d'un fichier vers Google Drive"""
    if not Path('/content/drive/MyDrive').exists():
        return

    try:
        drive_dir = Path(drive_backup_dir)
        drive_dir.mkdir(parents=True, exist_ok=True)
        dest_path = drive_dir / Path(file_path).name
        shutil.copy2(file_path, dest_path)
        size_mb = Path(file_path).stat().st_size / (1024**2)
        print(f"  ☁️ Backup Drive: {Path(file_path).name} ({size_mb:.1f} MB)")
    except Exception as e:
        print(f"  ⚠️ Backup Drive échoué: {e}")

# --- MAIN ---
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='data_baseline/data')
    parser.add_argument('--model_name', type=str, default='recobo/chemical-bert-uncased')
    parser.add_argument('--drive_backup_dir', type=str, default='/content/drive/MyDrive/dual_encoder_checkpoints')

    #  REPRISE D'ENTRAÎNEMENT
    parser.add_argument('--resume_from', type=str, required=True,
                       help='Chemin vers le checkpoint à reprendre (ex: dual_FINAL_..._ep150.pt)')
    parser.add_argument('--total_epochs', type=int, default=250,
                       help='Nombre total d\'epochs voulu (ex: 250 pour continuer de 150 à 250)')

    parser.add_argument('--lr_gnn', type=float, default=8e-4)
    parser.add_argument('--lr_bert', type=float, default=3e-5)
    parser.add_argument('--weight_decay', type=float, default=1e-4)
    parser.add_argument('--freeze_layers', type=int, default=0)
    parser.add_argument('--margin', type=float, default=0.2)
    parser.add_argument('--batch_size', type=int, default=16)
    parser.add_argument('--grad_accum', type=int, default=8)

    args = parser.parse_args()
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    # === 1. CHARGEMENT DU CHECKPOINT ===
    print("=" * 70)
    print("🔄 REPRISE D'ENTRAÎNEMENT - DUAL ENCODER (8 heads)")
    print("=" * 70)
    print(f"📥 Chargement du checkpoint : {args.resume_from}")

    checkpoint = torch.load(args.resume_from, map_location=DEVICE)

    # Extraire les infos du checkpoint
    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        state_dict = checkpoint['model_state_dict']
        saved_args = checkpoint.get('args', {})
        start_epoch = checkpoint.get('epoch', 0)

        # Utiliser les hyperparamètres sauvegardés
        args.lr_gnn = saved_args.get('lr_gnn', args.lr_gnn)
        args.lr_bert = saved_args.get('lr_bert', args.lr_bert)
        args.weight_decay = saved_args.get('weight_decay', args.weight_decay)
        args.freeze_layers = saved_args.get('freeze_layers', args.freeze_layers)
        args.margin = saved_args.get('margin', args.margin)
        args.batch_size = saved_args.get('batch_size', args.batch_size)
        args.grad_accum = saved_args.get('grad_accum', args.grad_accum)

        print(f"✅ Checkpoint chargé : Epoch {start_epoch}")
        print(f"   Loss: {checkpoint.get('train_loss', 'N/A')}")
    else:
        raise ValueError("Format de checkpoint non reconnu")

    drive_available = Path('/content/drive/MyDrive').exists()

    print("-" * 70)
    print(f"Device      : {DEVICE}")
    if drive_available:
        print(f"☁️ Google Drive: MONTÉ")
    print(f"LR GNN      : {args.lr_gnn}")
    print(f"LR BERT     : {args.lr_bert}")
    print(f"Weight Decay: {args.weight_decay}")
    print(f"Freeze Layers: {args.freeze_layers}")
    print(f"Margin      : {args.margin}")
    print(f"Batch Size  : {args.batch_size} × {args.grad_accum} = {args.batch_size * args.grad_accum}")
    print(f"📈 Reprise   : Epoch {start_epoch} → {args.total_epochs}")
    print("=" * 70)

    tokenizer = AutoTokenizer.from_pretrained(args.model_name)

    # === 2. DATASET (TRAIN+VAL) ===
    train_dataset = RawTextGraphDataset(Path(args.data_dir) / "train_graphs.pkl")
    val_dataset = RawTextGraphDataset(Path(args.data_dir) / "validation_graphs.pkl")
    combined_dataset = ConcatDataset([train_dataset, val_dataset])

    train_loader = DataLoader(
        combined_dataset,
        batch_size=args.batch_size,
        shuffle=True,
        collate_fn=DualCollate(tokenizer),
        num_workers=2,
        pin_memory=True
    )

    print(f"📊 Training sur {len(combined_dataset)} exemples (train: {len(train_dataset)}, val: {len(val_dataset)})")

    # === 3. MODÈLE (8 HEADS) ===
    gnn_config = {
        'hidden_dim': 256,
        'out_dim': 768,
        'num_layers': 4,
        'num_heads': 8,  # ← 8 heads (comme le checkpoint)
        'dropout': 0.1
    }

    model = DualEncoder(args.model_name, gnn_config, freeze_layers=args.freeze_layers).to(DEVICE)


    model.load_state_dict(state_dict)
    print("✅ Poids du modèle chargés")

    # === 4. OPTIMIZER & SCHEDULER ===
    optimizer = torch.optim.AdamW([
        {'params': model.graph_encoder.parameters(),
         'lr': args.lr_gnn,
         'weight_decay': args.weight_decay},
        {'params': model.text_encoder.parameters(),
         'lr': args.lr_bert,
         'weight_decay': args.weight_decay / 10},
        {'params': model.text_proj.parameters(),
         'lr': args.lr_bert,
         'weight_decay': args.weight_decay}
    ])

    #  NOUVEAU SCHEDULER pour la continuation
    # On crée un scheduler qui part de start_epoch
    remaining_epochs = args.total_epochs - start_epoch

    scheduler = OneCycleLR(
        optimizer,
        max_lr=[args.lr_gnn, args.lr_bert, args.lr_bert],
        steps_per_epoch=len(train_loader) // args.grad_accum,
        epochs=remaining_epochs,  # Epochs restants
        pct_start=0.05,  # Petit warmup pour reprendre doucement
        anneal_strategy='cos'
    )

    print(f"✅ Scheduler configuré pour {remaining_epochs} epochs supplémentaires")

    scaler = GradScaler()

    log_file = Path(args.data_dir) / "training_log_final_resumed.json"
    if log_file.exists():
        with open(log_file, 'r') as f:
            training_logs = json.load(f)
        print(f"📝 Logs existants chargés ({len(training_logs)} epochs)")
    else:
        training_logs = []

    print("\n🏋️ Reprise de l'entraînement...\n")

    # === 5. BOUCLE D'ENTRAÎNEMENT ===
    for epoch in range(start_epoch, args.total_epochs):
        model.train()
        total_loss = 0
        optimizer.zero_grad()

        for step, (graphs, text_inputs) in enumerate(train_loader):
            graphs = graphs.to(DEVICE)
            text_inputs = {k: v.to(DEVICE) for k, v in text_inputs.items()}

            with autocast():
                g_emb, t_emb = model(graphs, text_inputs)
                loss = triplet_loss(g_emb, t_emb, margin=args.margin) / args.grad_accum

            scaler.scale(loss).backward()

            if (step + 1) % args.grad_accum == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()

            total_loss += loss.item() * args.grad_accum

        avg_loss = total_loss / len(train_loader)

        # Log
        log_entry = {
            'epoch': epoch + 1,
            'train_loss': avg_loss
        }
        training_logs.append(log_entry)

        print(f"Epoch {epoch+1}/{args.total_epochs} | Train Loss: {avg_loss:.4f}")

        if (epoch + 1) % 5 == 0 or epoch == args.total_epochs - 1:
            effective_bs = args.batch_size * args.grad_accum
            save_name = (f"dual_FINAL_RESUMED_lrGNN{args.lr_gnn}_lrBERT{args.lr_bert}_"
                        f"wd{args.weight_decay}_frz{args.freeze_layers}_"
                        f"margin{args.margin}_bs{effective_bs}_ep{epoch+1}.pt")

            checkpoint_path = Path(args.data_dir) / save_name
            torch.save({
                'model_state_dict': model.state_dict(),
                'args': vars(args),
                'epoch': epoch + 1,
                'train_loss': avg_loss
            }, checkpoint_path)

            print(f"  💾 Checkpoint sauvegardé : {save_name}")

            if drive_available:
                backup_to_drive(checkpoint_path, args.drive_backup_dir)

        with open(log_file, 'w') as f:
            json.dump(training_logs, f, indent=2)

        if drive_available and ((epoch + 1) % 10 == 0 or epoch == args.total_epochs - 1):
            backup_to_drive(log_file, args.drive_backup_dir)

    print("\n" + "=" * 70)
    print("✅ Entraînement terminé !")
    print(f"📝 Logs sauvegardés dans : {log_file}")
    if drive_available:
        print(f"☁️ Backups disponibles sur : {args.drive_backup_dir}")
    print("=" * 70)

if __name__ == "__main__":
    main()

Writing data_baseline/resume_training_dual_encoder.py


In [ ]:
!python data_baseline/resume_training_dual_encoder.py \
    --resume_from data_baseline/data/dual_FINAL_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128_ep150.pt \
    --total_epochs 250 \
    --data_dir data_baseline/data \
    --drive_backup_dir /content/drive/MyDrive/Altegrad_Results/dual_encoder_checkpoints

🔄 REPRISE D'ENTRAÎNEMENT - DUAL ENCODER (8 heads)
📥 Chargement du checkpoint : data_baseline/data/dual_FINAL_lrGNN0.0008_lrBERT3e-05_wd0.0001_frz0_margin0.2_bs128_ep150.pt
✅ Checkpoint chargé : Epoch 150
   Loss: 0.0013596335987279798
----------------------------------------------------------------------
Device      : cuda
☁️ Google Drive: MONTÉ
LR GNN      : 0.0008
LR BERT     : 3e-05
Weight Decay: 0.0001
Freeze Layers: 0
Margin      : 0.2
Batch Size  : 16 × 8 = 128
📈 Reprise   : Epoch 150 → 250
📊 Training sur 32008 exemples (train: 31008, val: 1000)
2026-01-12 19:17:40.277973: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768245460.304316  127352 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768245460.311584  127352 cuda_blas.cc:1407] 

_____